In [ ]:
# ------------------------------------------------------------------------
#
# TITLE - plot_stellar_sample.ipynb
# AUTHOR - James Lane
# PROJECT - ges-mass
#
# ------------------------------------------------------------------------
#
# Docstrings and metadata:
'''Make figure plots of the stellar sample. Position, chemistry, and kinematics 
are shown.
'''

__author__ = "James Lane"

In [ ]:
### Imports

## Basic
import numpy as np, pdb, sys, os, numbers, dill as pickle, copy
import matplotlib.pyplot as plt
from matplotlib import patches
from matplotlib import path
import astropy.units as apu
import scipy.stats

## galpy
from galpy import orbit
from galpy import potential
from galpy import actionAngle as aA

sys.path.append('../../src/')
from ges_mass import plot as pplot
from ges_mass import util as putil

### Notebook setup

%matplotlib inline
plt.style.use('../../src/mpl/project.mplstyle')
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

### Colors setup

project_colors = pplot.colors()
rainbow_cmap = project_colors.colourmap('rainbow')

### Keywords, Pathing, Loading, Data Preparation

In [ ]:
## Keywords
cdict = putil.load_config_to_dict()
keywords = ['BASE_DIR','APOGEE_DR','APOGEE_RESULTS_VERS','GAIA_DR','NDMOD',
            'DMOD_MIN','DMOD_MAX','LOGG_MIN','LOGG_MAX','FEH_MIN','FEH_MAX',
            'FEH_MIN_GSE','FEH_MAX_GSE','NPROCS','DF_VERSION','KSF_VERSION',
            'RO','VO','ZO']
base_dir,apogee_dr,apogee_results_vers,gaia_dr,ndmod,dmod_min,dmod_max,\
    logg_min,logg_max,feh_min,feh_max,feh_min_gse,feh_max_gse,nprocs,\
    df_version,ksf_version,ro,vo,zo = putil.parse_config_dict(cdict,keywords)
logg_range = [logg_min,logg_max]
feh_range = [feh_min,feh_max]
feh_range_gse = [feh_min_gse,feh_max_gse]

## Pathing
fit_paths = putil.prepare_paths(base_dir,apogee_dr,apogee_results_vers,gaia_dr,
                                df_version,ksf_version)
data_dir,version_dir,ga_dir,gap_dir,_,_,_ = fit_paths
os.makedirs(gap_dir,exist_ok=True)
fig_dir = './fig/'

## Filenames
sample_kinematics = True
if sample_kinematics:
    omask_kinematics_filename = gap_dir+'clean_kinematics_sampled.npy'
else:
    omask_kinematics_filename = gap_dir+'clean_kinematics_no_sample.npy'

## Potential
mwpot = potential.MWPotential2014
potential.turn_physical_on(mwpot,ro=ro,vo=vo)
phi0 = potential.evaluatePotentials(mwpot,1e10,0).value

# Functions
def line_equation(x,m,b):
    return m*x+b

### Abundance lines (for masking), axis limits, decoration parameters

In [ ]:
# Halo separation from Lane+2022, also used in Mackereth+2019
alpha_fe_halo_line_x = [-1.2,-0.9]
alpha_fe_halo_line_y = [0.5,-0.2]
alpha_fe_halo_line_m, alpha_fe_halo_line_b = pplot.get_params_from_line(
    alpha_fe_halo_line_x, alpha_fe_halo_line_y)

# # Double GSE/thick disk separation for this work?
# alpha_fe_halo_line_broken_m1, alpha_fe_halo_line_broken_b1 = -0.21,0.08
# alpha_fe_halo_line_broken_m2, alpha_fe_halo_line_broken_b2 = -0.95,-0.58
# alpha_fe_halo_line_broken_xint =\
#  (alpha_fe_halo_line_broken_b1-alpha_fe_halo_line_broken_b2)/\
#  (alpha_fe_halo_line_broken_m2-alpha_fe_halo_line_broken_m1)

# [Mg/Mn]-[Al/Fe] Halo separation from Horta+21
mg_mn_al_fe_halo_line_x = [-9999,-0.06,0.18]
mg_mn_al_fe_halo_line_y = [0.25,0.25,1.3]
mg_mn_al_fe_halo_line_m, mg_mn_al_fe_halo_line_b = pplot.get_params_from_line(
    mg_mn_al_fe_halo_line_x[1:], mg_mn_al_fe_halo_line_y[1:])

# [Al/Fe] Halo separation from Belokurov+22
al_fe_halo_line_x_all = [-1., -1., -0.6, -0.6]
al_fe_halo_line_y_all = [9999., -0.1, -0.3, -9999.]
al_fe_halo_line_x_acc = [-9999., -1., -0.6, -0.6]
al_fe_halo_line_y_acc = [-0.1, -0.1, -0.3, -9999.]
al_fe_halo_line_x_ext = [-9999., -0.6, -0.6]
al_fe_halo_line_y_ext = [-0.1, -0.1, -0.3]
# al_fe_halo_line_x = [-9999,-0.6,-0.6]
# al_fe_halo_line_y = [-0.075,-0.075,-9999]

# Line weights and styles
c_primary = 'Black'
c_secondary = 'Black'
lw_primary = 1.5
lw_secondary = 1.
ls_primary = 'dashed'
ls_secondary = 'solid'
alpha_primary = 1.
alpha_secondary = 0.5
zorder_primary = 6.
zorder_secondary = 6.
lw_histogram_primary = 3.
lw_histogram_secondary = 2.
lw_histogram_kinematic = 1.
zorder_histogram_primary = 2.
zorder_histogram_secondary = 3.
zorder_histogram_kinematic = 4.

# Abundance axis limits
feh_lims = [-2.5,1.0]
alfe_lims = [-0.8,0.8]
mgfe_lims = [-0.3,0.6]
mgmn_lims = [-0.6,1.3]

# Contour levels
contour_levels = [10**1.75,10**2.25,10**2.75,10**3.25]

# Labelling
halo_type_labels = ['GS/E','Other halo']
halo_type_label_fs = 8
xs_halo_type_label = [0.03,0.05]
ys_halo_type_label = [0.03,0.925]

### Get data

In [ ]:
# Cleaned kinematics
if os.path.exists(omask_kinematics_filename):
    print('Loading cleaned kinematics from '+omask_kinematics_filename)
    with open(omask_kinematics_filename,'rb') as f:
        omask_kinematics = pickle.load(f)
else:
    sys.exit('Clean kinematics first')

gaia_omask,allstar_omask,os_omask,eELzs_omask,accs_omask,orbextr_omask = \
    omask_kinematics

# Unpack the orbital extrema
zmax_omask,rperi_omask,rapo_omask = orbextr_omask

### Define all masks

Note that the sample has already been masked according to:
- Measured Gaia parameters and APOGEE RVs
- Fractional weighted distance uncertainty < 0.2
- log(g) error < 0.1
- [Fe/H] and [Al/Fe] measured

So adding the log(g) mask should give the 'quality sample'

In [ ]:
# log(g) mask
logg_mask = (allstar_omask['LOGG'] > logg_min) &\
            (allstar_omask['LOGG'] < logg_max)

In [ ]:
print('Number of stars in the quality sample')
print(np.sum(logg_mask))

In [ ]:
# Other chemistry masks, not for use here just for info
mg_fe_mask = allstar_omask['MG_FE'] > -9999
mn_fe_mask = allstar_omask['MN_FE'] > -9999

In [ ]:
print(np.sum(mg_fe_mask))
print(np.sum(mn_fe_mask))
print(np.sum(mg_fe_mask&logg_mask))
print(np.sum(mn_fe_mask&logg_mask))

In [ ]:
# [Al/Fe] mask and [Fe/H] mask
omask_feh,_ = putil.get_metallicity(allstar_omask,'FE_H')
omask_alfe,_ = putil.get_metallicity(allstar_omask,'AL_FE')
abund_mask = putil.lines_boundary_mask(omask_feh, omask_alfe, 
                                       al_fe_halo_line_x_all, 
                                       al_fe_halo_line_y_all, where='left')

The 'halo sample' is the quality sample masked based on [Al/Fe] and [Fe/H] abundances

In [ ]:
print('Number of stars in the halo sample')
print(np.sum(logg_mask&abund_mask))

In [ ]:
# Make kinematic masks
kmasks = {}
halo_selection_survey_dict = putil.lane2022_kinematic_selections(
    version='current')
kin_spaces = ['eLz','AD','JRLz']
n_kin_spaces = len(kin_spaces)

# _os_kmask = os_omask[abund_mask & logg_mask]
# _eELzs_kmask = eELzs_omask[:,(abund_mask & logg_mask)]
# _accs_kmask = accs_omask[:,(abund_mask & logg_mask)]

for i in range(n_kin_spaces):
    xs,ys = pplot.get_plottable_data( [os_omask,], [eELzs_omask,], 
        [accs_omask,], np.array([1,]), kin_spaces[i], phi0=phi0, 
        absolute=True)
    this_selection = halo_selection_survey_dict[kin_spaces[i]]
    selec_mask = pplot.is_in_scaled_selection(xs, ys, this_selection, 
        factor=[1.,1.])
    kmasks.update({kin_spaces[i]:selec_mask})

### Plotting information

In [ ]:
# Column-sized figure width
columnwidth = 244./72.27 # In inches, from pt
# Full-sized figure width
textwidth = 508./72.27 # In inches, from pt

### Plot the spatial distribution of the sample

In [ ]:
# Parameters
n_bins = 20
r_range = (0,40)
log = True
label_fs = 12
legend_fs = 9

fig = plt.figure(figsize=(columnwidth,3))
ax = fig.add_subplot(111)

hmask = logg_mask & abund_mask
rs = os_omask.r().to(apu.kpc).value

# Plot the whole quality sample
ax.hist(rs[logg_mask], bins=n_bins, range=r_range, histtype='step', 
        color='Grey', lw=lw_histogram_primary, log=True, 
        zorder=zorder_histogram_primary, label='Quality sample')

# Plot the whole halo sample
ax.hist(rs[hmask], bins=n_bins, range=r_range, histtype='step', color='Black', 
        lw=lw_histogram_secondary, ls='dashed', log=True, 
        zorder=zorder_histogram_secondary, label='Halo sample')

# Plot the kinematic selections
colors = ['Red','DarkOrange','DodgerBlue']
labels = ['$e-L_\mathrm{z}$','AD','$\sqrt{J_\mathrm{R}}-L_\mathrm{z}$']
for i in range(n_kin_spaces):
    ax.hist(rs[hmask & kmasks[kin_spaces[i]]], bins=n_bins, range=r_range, 
            histtype='step', color=colors[i], lw=lw_histogram_kinematic, 
            log=True, zorder=zorder_histogram_kinematic, label=labels[i])

ax.legend(fontsize=legend_fs)
ax.set_xlabel('r [kpc]', fontsize=label_fs)
ax.set_ylabel('N', fontsize=label_fs)

fig.tight_layout()
fig.subplots_adjust(left=0.15, bottom=0.15,
                    top=0.95, right=0.95)
fig.savefig(fig_dir+'radius_histogram.pdf')
fig.show()

### Plot the whole halo sample

In [ ]:
# Paramters
n_bins = 50
label_fs = 12
ylabel_pad = -2
cbar_ticklabel_fs = 8
ticklabel_fs = 9
legend_fs = 9

fig = plt.figure(figsize=(textwidth,2.5))
axs = fig.subplots(nrows=1, ncols=3).flatten()

abunds_x = ['FE_H','FE_H','AL_FE']
abunds_y = ['AL_FE','MG_FE','MG_MN']
abunds_xlabels = ['[Fe/H]', '[Fe/H]', '[Al/Fe]']
abunds_ylabels = ['[Al/Fe]', '[Mg/Fe]', '[Mg/Mn]']
xlims = [ feh_lims, feh_lims, alfe_lims]
ylims = [ alfe_lims, mgfe_lims, mgmn_lims]

# ecc_sort = np.argsort(eELzs_omask[0])

hmask = logg_mask & abund_mask

for i in range(len(axs)):
    # Quality sample
    xa,_ = putil.get_metallicity(allstar_omask[logg_mask],abunds_x[i])
    ya,_ = putil.get_metallicity(allstar_omask[logg_mask],abunds_y[i])
    # Halo sample
    xh,_ = putil.get_metallicity(allstar_omask[hmask],abunds_x[i])
    yh,_ = putil.get_metallicity(allstar_omask[hmask],abunds_y[i])
    
    # Do the eccentricity sorting based on the masked data
    ecc_sort_mask = np.argsort(eELzs_omask[0][hmask])
    
    # Plot both sets of points, halo (colour) and other (gray)
    pts = axs[i].scatter(xh[ecc_sort_mask], 
                         yh[ecc_sort_mask], 
                         c=eELzs_omask[0][hmask][ecc_sort_mask], 
                         cmap=rainbow_cmap, edgecolors='None', vmin=0, vmax=1, 
                         marker='o', s=5, zorder=4, rasterized=True)
    axs[i].scatter(xa, ya, c='Grey', 
                   edgecolors='None', marker='o', s=2, 
                   zorder=3, rasterized=True)
    
    # Add contours
    xbinsize = (xlims[i][1]-xlims[i][0])/n_bins
    ybinsize = (ylims[i][1]-ylims[i][0])/n_bins
    hist,xedge,yedge = np.histogram2d(xa,ya,bins=n_bins,
                                      range=[[xlims[i][0]-xbinsize,
                                              xlims[i][1]+xbinsize],
                                             [ylims[i][0]-ybinsize,
                                              ylims[i][1]+ybinsize]])
    xcents = xedge[:-1] + (xlims[i][1]-xlims[i][0])/n_bins
    ycents = yedge[:-1] + (ylims[i][1]-ylims[i][0])/n_bins
    xcnt,ycnt = np.meshgrid(xcents,ycents)
    axs[i].contour(xcnt, ycnt, hist.T, colors='Black', levels=[50,100,500], 
                   zorder=5, alpha=0.5)
     
    cax = fig.add_axes([0.9,0.24,0.02,0.64])
    cbar = fig.colorbar(pts, cax=cax)
    cbar.ax.tick_params(labelsize=cbar_ticklabel_fs) 
    cbar.set_label(r'eccentricity', fontsize=label_fs)
    
    axs[i].set_xlabel(abunds_xlabels[i], fontsize=label_fs)
    axs[i].set_ylabel(abunds_ylabels[i], fontsize=label_fs, labelpad=ylabel_pad)
    axs[i].set_xlim(xlims[i])
    axs[i].set_ylim(ylims[i])
    axs[i].tick_params(labelsize=ticklabel_fs)
    
    # Add lines
    if abunds_x[i] == 'FE_H' and abunds_y[i] == 'MG_FE':
        axs[i].plot(np.arange(-3,2,1.), 
                    pplot.line_equation(np.arange(-3,2,1.),
                                        alpha_fe_halo_line_m,
                                        alpha_fe_halo_line_b), 
                    linestyle=ls_secondary, color=c_secondary, 
                    linewidth=lw_secondary, zorder=zorder_secondary, 
                    alpha=alpha_secondary)
        axs[i].axvline(-1, linestyle=ls_secondary, color=c_secondary, 
                       linewidth=lw_secondary, zorder=zorder_secondary,
                       alpha=alpha_secondary)
    if abunds_x[i] == 'AL_FE' and abunds_y[i] == 'MG_MN':
        axs[i].axhline(mg_mn_al_fe_halo_line_y[0], linestyle=ls_secondary, 
                       color=c_secondary, linewidth=lw_secondary, 
                       zorder=zorder_secondary, alpha=alpha_secondary)
        axs[i].plot([mg_mn_al_fe_halo_line_x[1],mg_mn_al_fe_halo_line_x[2]],
                    [mg_mn_al_fe_halo_line_y[1],mg_mn_al_fe_halo_line_y[2]],
                    linestyle=ls_secondary, color=c_secondary, 
                    linewidth=lw_secondary, zorder=zorder_secondary, 
                    alpha=alpha_secondary)
    if abunds_x[i] == 'FE_H' and abunds_y[i] == 'AL_FE':
        for j in range(len(al_fe_halo_line_x_all)-1):
            axs[i].plot([al_fe_halo_line_x_all[j],al_fe_halo_line_x_all[j+1]],
                        [al_fe_halo_line_y_all[j],al_fe_halo_line_y_all[j+1]],
                        linestyle=ls_primary, color='Black', 
                        linewidth=lw_primary, zorder=zorder_primary, 
                        alpha=alpha_primary)
        for j in range(2):
            axs[i].plot([al_fe_halo_line_x_acc[j],al_fe_halo_line_x_acc[j+1]],
                        [al_fe_halo_line_y_acc[j],al_fe_halo_line_y_acc[j+1]],
                        linestyle=ls_primary, color='Black', 
                        linewidth=lw_primary, zorder=zorder_primary, 
                        alpha=alpha_primary)
        # Label the spaces
        for j in range(len(halo_type_labels)):     
            axs[i].text(xs_halo_type_label[j], ys_halo_type_label[j],
                        halo_type_labels[j], transform=axs[i].transAxes, 
                        fontsize=halo_type_label_fs)

        # for j in range(len(al_fe_halo_line_x_ext)-1):
        #     axs[i].plot([al_fe_halo_line_x_ext[j],al_fe_halo_line_x_ext[j+1]],
        #                 [al_fe_halo_line_y_ext[j],al_fe_halo_line_y_ext[j+1]],
        #                 linestyle='dotted', color='Black', linewidth=1., 
        #                 zorder=6)
    
fig.tight_layout()
fig.subplots_adjust(wspace=0.35, left=0.075, right=0.88)
fig.savefig(fig_dir+'halo_abundances.pdf', dpi=400)
fig.show()

### Plot the kinematics of the samples

In [ ]:
# Parameters
AD_dedge = 1.2
n_bins = 40
label_fs = 12
ylabel_pad = -2
cbar_ticklabel_fs = 8
ticklabel_fs = 9
legend_fs = 9

# Limits dictionary. [ [xmin,xmax], [ymin,ymax] ]
lims_dict = {'JRLz':   [[-4500,4500], [0,80]],
             'eLz':    [[-4500,4500], [0.,1.0]],
             'AD':     [[-AD_dedge,AD_dedge], [-AD_dedge,AD_dedge]]
            }
# Labels dictionary. [xlabel, ylabel]
labels_dict = {'JRLz':   [r'$L_\mathrm{z}$',r'$\sqrt{J_\mathrm{R}}$'],
               'eLz':    [r'$L_\mathrm{z}$',r'eccentricity'],
               'AD':     [r'$L_\mathrm{z}/J_\mathrm{tot}$',
                          r'$(J_\mathrm{z}-J_\mathrm{R})/J_\mathrm{tot}$']
               }
# Selection plotting keywords
halo_selection_kws = {'linestyle':'solid', 'linewidth':2., 'facecolor':'None', 
                      'edgecolor':'Black', 'color':'Black', 
                      'label':r'High-$\beta$', 'zorder':4, 'alpha':0.5}

fig = plt.figure(figsize=(textwidth,2.5))
axs = fig.subplots(nrows=1, ncols=3)

# To mask halo sample
hmask = logg_mask & abund_mask
# For coloring scatter plot
halo_feh = allstar_omask['FE_H'][hmask]
halo_feh_sort = np.argsort(halo_feh)

for i in range(n_kin_spaces):
    
    # Limits and labels
    xlim,ylim = lims_dict[kin_spaces[i]]
    xlabel,ylabel = labels_dict[kin_spaces[i]]

    # Quality sample
    xa,ya = pplot.get_plottable_data( [os_omask[logg_mask],], 
        [eELzs_omask[:,logg_mask],], [accs_omask[:,logg_mask],], 
        np.array([1,]), kin_spaces[i], phi0=phi0, 
        absolute=True)
    # Halo sample
    xh,yh = pplot.get_plottable_data( [os_omask[hmask],], 
        [eELzs_omask[:,hmask],], [accs_omask[:,hmask],], np.array([1,]), 
        kin_spaces[i], phi0=phi0, absolute=True)
    
    # 2D histogram of quality sample
    hist = np.histogram2d(xa,ya,bins=n_bins,
                          range=[[xlim[0],xlim[1]],[ylim[0],ylim[1]]])[0]
    axs[i].imshow(np.log10(np.rot90(hist)), cmap='Greys', vmin=-0.5, 
                  vmax=3.5, aspect='auto', zorder=2,
                  extent=(xlim[0],xlim[1],ylim[0],ylim[1]), rasterized=True)
    
    # Scatter plot of halo sample
    pts = axs[i].scatter(xh[halo_feh_sort], yh[halo_feh_sort], marker='o', s=2,
                         c=halo_feh[halo_feh_sort], zorder=3, cmap=rainbow_cmap, 
                         vmin=-3.0,vmax=-0.5, rasterized=True)

    # Decoration
    pplot.axis_limits_and_labels(axs[i], xlim, ylim, xlabel, ylabel, 
        mixture_text=None, is_left_edge=True, is_top_edge=False, 
        label_fontsize=label_fs)
    if kin_spaces[i] == 'AD':
        pplot.add_diamond_boundary(axs[i], dedge=AD_dedge, draw_tick_fac=50.)
    pplot.add_selection_boundaries(axs[i], 
        halo_selection_survey_dict[kin_spaces[i]], 
        plot_kwargs=halo_selection_kws, plot_cent=False)
    
    axs[i].tick_params(labelsize=ticklabel_fs)
    
    if i == 0:
        cax = fig.add_axes([0.9,0.22,0.02,0.66])
        cbar = fig.colorbar(pts, cax=cax)
        cbar.ax.tick_params(labelsize=cbar_ticklabel_fs) 
        cbar.set_label(r'[Fe/H]', fontsize=label_fs)

fig.tight_layout()
fig.subplots_adjust(wspace=0.4, left=0.075, right=0.88)
fig.savefig(fig_dir+'halo_kinematics.pdf', dpi=400)
fig.show()

### Plot the chemistry of the kinematic selections

### Just [Al/Fe] vs [Fe/H] and [Mg/Fe] vs [Fe/H]

In [ ]:
# Paramters
n_bins = 50
label_fs = 12
ylabel_pad = -2
cbar_ticklabel_fs = 8
ticklabel_fs = 9
legend_fs = 9

n_abunds = 2
fig = plt.figure(figsize=(textwidth,5))
axs = fig.subplots(nrows=n_abunds, ncols=3)

abunds_x = ['FE_H','FE_H']
abunds_y = ['AL_FE','MG_FE']
abunds_xlabels = ['[Fe/H]', '[Fe/H]']
abunds_ylabels = ['[Al/Fe]', '[Mg/Fe]']
xlims = [ feh_lims, feh_lims]
ylims = [ alfe_lims, mgfe_lims]

# ecc_sort = np.argsort(eELzs_omask[0])
acc_masks = []

for i in range(n_abunds):
    # Abundances
    xa,_ = putil.get_metallicity(allstar_omask[logg_mask],abunds_x[i])
    ya,_ = putil.get_metallicity(allstar_omask[logg_mask],abunds_y[i])

    for j in range(n_kin_spaces):
        # Make the kinematic space mask
        ksamp_mask = logg_mask & abund_mask & kmasks[kin_spaces[j]]
        
        xk,_ = putil.get_metallicity(allstar_omask[ksamp_mask],abunds_x[i])
        yk,_ = putil.get_metallicity(allstar_omask[ksamp_mask],abunds_y[i])

        # Make the accreted mask in the 
        if abunds_x[i]=='FE_H' and abunds_y[i]=='AL_FE':
            acc_mask = putil.lines_boundary_mask(xk, yk,
                al_fe_halo_line_x_acc, al_fe_halo_line_y_acc, where='left')
            acc_masks.append(acc_mask)
            print('Number of stars in kin space, low [Al/Fe]: '+str(kin_spaces[j]), np.sum(acc_mask))
            print('Number of stars in kin space, high [Al/Fe]: '+str(kin_spaces[j]), np.sum(~acc_mask))

        # Plot
        axs[i,j].scatter(xk[acc_masks[j]], yk[acc_masks[j]], c=colors[j],  
            edgecolors='Black', marker='o', s=10, linewidth=0.5, zorder=4, 
            rasterized=True)
        axs[i,j].scatter(xk[~acc_masks[j]], yk[~acc_masks[j]], c='Black', 
            marker='x', s=10, linewidth=1., zorder=5, rasterized=True)
        axs[i,j].scatter(xa, ya, c='Grey', edgecolors='None', marker='o', s=2, 
            zorder=3, rasterized=True)
        
        # Add contours
        xbinsize = (xlims[i][1]-xlims[i][0])/n_bins
        ybinsize = (ylims[i][1]-ylims[i][0])/n_bins
        hist,xedge,yedge = np.histogram2d(xa,ya,
                                          bins=n_bins,
                                          range=[[xlims[i][0]-xbinsize,
                                                  xlims[i][1]+xbinsize],
                                                 [ylims[i][0]-ybinsize,
                                                  ylims[i][1]+ybinsize]])
        xcents = xedge[:-1] + (xlims[i][1]-xlims[i][0])/n_bins
        ycents = yedge[:-1] + (ylims[i][1]-ylims[i][0])/n_bins
        xcnt,ycnt = np.meshgrid(xcents,ycents)
        axs[i,j].contour(xcnt, ycnt, hist.T, colors='Black', 
                         levels=[10**1.75,10**2.25,10**2.75,10**3.25], 
                         zorder=5, alpha=0.5)
        
        axs[i,j].set_xlabel(abunds_xlabels[i], fontsize=label_fs)
        axs[i,j].set_ylabel(abunds_ylabels[i], fontsize=label_fs)
        axs[i,j].set_xlim(xlims[i])
        axs[i,j].set_ylim(ylims[i])
        axs[i,j].tick_params(labelsize=ticklabel_fs)
        
        # Add lines
        if abunds_x[i] == 'FE_H' and abunds_y[i] == 'MG_FE':
            axs[i,j].plot(np.arange(-3,2,1.), 
                        pplot.line_equation(np.arange(-3,2,1.),
                                            alpha_fe_halo_line_m,
                                            alpha_fe_halo_line_b), 
                        linestyle=ls_secondary, color=c_secondary, 
                        linewidth=lw_secondary, zorder=zorder_secondary, 
                        alpha=alpha_secondary)
            axs[i,j].axvline(-1, linestyle=ls_secondary, color=c_secondary, 
                        linewidth=lw_secondary, zorder=zorder_secondary,
                        alpha=alpha_secondary)
        if abunds_x[i] == 'AL_FE' and abunds_y[i] == 'MG_MN':
            axs[i,j].axhline(mg_mn_al_fe_halo_line_y[0], linestyle=ls_secondary, 
                        color=c_secondary, linewidth=lw_secondary, 
                        zorder=zorder_secondary, alpha=alpha_secondary)
            axs[i,j].plot([mg_mn_al_fe_halo_line_x[1],mg_mn_al_fe_halo_line_x[2]],
                        [mg_mn_al_fe_halo_line_y[1],mg_mn_al_fe_halo_line_y[2]],
                        linestyle=ls_secondary, color=c_secondary, 
                        linewidth=lw_secondary, zorder=zorder_secondary, 
                        alpha=alpha_secondary)
        if abunds_x[i] == 'FE_H' and abunds_y[i] == 'AL_FE':
            for k in range(len(al_fe_halo_line_x_all)-1):
                axs[i,j].plot([al_fe_halo_line_x_all[k],al_fe_halo_line_x_all[k+1]],
                            [al_fe_halo_line_y_all[k],al_fe_halo_line_y_all[k+1]],
                            linestyle=ls_primary, color='Black', 
                            linewidth=lw_primary, zorder=zorder_primary, 
                            alpha=alpha_primary)
            for k in range(2):
                axs[i,j].plot([al_fe_halo_line_x_acc[k],al_fe_halo_line_x_acc[k+1]],
                            [al_fe_halo_line_y_acc[k],al_fe_halo_line_y_acc[k+1]],
                            linestyle=ls_primary, color='Black', 
                            linewidth=lw_primary, zorder=zorder_primary, 
                            alpha=alpha_primary)
            # Label the spaces
            if j == 0:
                for k in range(len(halo_type_labels)):     
                    axs[i,j].text(xs_halo_type_label[k], ys_halo_type_label[k],
                                halo_type_labels[k], transform=axs[i,j].transAxes, 
                                fontsize=halo_type_label_fs)
    
        # Remove labels
        if j != 0:
            axs[i,j].set_yticklabels([])
            axs[i,j].set_ylabel('')
        #if i == 0:
        #    axs[i,j].set_xticklabels([])
        #    axs[i,j].set_xlabel('')
    
        # Add titles for each subplot
        if i == 0:
            axs[i,j].set_title(labels[j], fontsize=label_fs)
            # axs[i,j].text(0.7,0.95, labels[j], fontsize=20, 
            #             transform=axs[i,j].transAxes, va='top', ha='left')

fig.tight_layout()
fig.subplots_adjust(hspace=0.35, wspace=0.2, left=0.09, right=0.9)
fig.savefig(fig_dir+'kinematic_sample_abundances.pdf', dpi=400)
fig.show()

### Same plot but with marginal distributions

In [ ]:
# Paramters
n_bins = 50
label_fs = 12
ylabel_pad = -2
cbar_ticklabel_fs = 8
ticklabel_fs = 9
legend_fs = 9

n_abunds = 2
fig = plt.figure(figsize=(textwidth,5))
gs = fig.add_gridspec(nrows=n_abunds+1, ncols=4, width_ratios=[1,1,1,0.33],
    height_ratios=[0.33,1,1])
axs = np.empty((n_abunds,3), dtype=object)
for i in range(3):
    axs[0,i] = fig.add_subplot(gs[1,i])
    axs[1,i] = fig.add_subplot(gs[2,i])
ax_feh = fig.add_subplot(gs[0,1])
ax_alfe = fig.add_subplot(gs[1,-1])
ax_mgfe = fig.add_subplot(gs[2,-1])  

abunds_x = ['FE_H','FE_H']
abunds_y = ['AL_FE','MG_FE']
abunds_xlabels = ['[Fe/H]', '[Fe/H]']
abunds_ylabels = ['[Al/Fe]', '[Mg/Fe]']
xlims = [ feh_lims, feh_lims]
ylims = [ alfe_lims, mgfe_lims]

# ecc_sort = np.argsort(eELzs_omask[0])
acc_masks = []

for i in range(n_abunds):
    # Abundances
    xa,_ = putil.get_metallicity(allstar_omask[logg_mask],abunds_x[i])
    ya,_ = putil.get_metallicity(allstar_omask[logg_mask],abunds_y[i])

    for j in range(n_kin_spaces):
        # Make the kinematic space mask
        ksamp_mask = logg_mask & abund_mask & kmasks[kin_spaces[j]]
        
        xk,_ = putil.get_metallicity(allstar_omask[ksamp_mask],abunds_x[i])
        yk,_ = putil.get_metallicity(allstar_omask[ksamp_mask],abunds_y[i])

        # Make the accreted mask in the 
        if abunds_x[i]=='FE_H' and abunds_y[i]=='AL_FE':
            acc_mask = putil.lines_boundary_mask(xk, yk,
                al_fe_halo_line_x_acc, al_fe_halo_line_y_acc, where='left')
            acc_masks.append(acc_mask)
            print('Number of stars in kin space, low [Al/Fe]: '+str(kin_spaces[j]), np.sum(acc_mask))
            print('Number of stars in kin space, high [Al/Fe]: '+str(kin_spaces[j]), np.sum(~acc_mask))

        # Plot
        axs[i,j].scatter(xk[acc_masks[j]], yk[acc_masks[j]], c=colors[j],  
            edgecolors='Black', marker='o', s=10, linewidth=0.5, zorder=4, 
            rasterized=True)
        axs[i,j].scatter(xk[~acc_masks[j]], yk[~acc_masks[j]], c='Black', 
            marker='x', s=10, linewidth=1., zorder=5, rasterized=True)
        axs[i,j].scatter(xa, ya, c='Grey', edgecolors='None', marker='o', s=2, 
            zorder=3, rasterized=True)
        
        # Add contours
        xbinsize = (xlims[i][1]-xlims[i][0])/n_bins
        ybinsize = (ylims[i][1]-ylims[i][0])/n_bins
        hist,xedge,yedge = np.histogram2d(xa,ya,
                                          bins=n_bins,
                                          range=[[xlims[i][0]-xbinsize,
                                                  xlims[i][1]+xbinsize],
                                                 [ylims[i][0]-ybinsize,
                                                  ylims[i][1]+ybinsize]])
        xcents = xedge[:-1] + (xlims[i][1]-xlims[i][0])/n_bins
        ycents = yedge[:-1] + (ylims[i][1]-ylims[i][0])/n_bins
        xcnt,ycnt = np.meshgrid(xcents,ycents)
        axs[i,j].contour(xcnt, ycnt, hist.T, colors='Black', 
                         levels=[10**1.75,10**2.25,10**2.75,10**3.25], 
                         zorder=5, alpha=0.5)
        
        axs[i,j].set_xlabel(abunds_xlabels[i], fontsize=label_fs)
        axs[i,j].set_ylabel(abunds_ylabels[i], fontsize=label_fs)
        axs[i,j].set_xlim(xlims[i])
        axs[i,j].set_ylim(ylims[i])
        axs[i,j].tick_params(labelsize=ticklabel_fs)
        
        # Add lines
        if abunds_x[i] == 'FE_H' and abunds_y[i] == 'MG_FE':
            axs[i,j].plot(np.arange(-3,2,1.), 
                        pplot.line_equation(np.arange(-3,2,1.),
                                            alpha_fe_halo_line_m,
                                            alpha_fe_halo_line_b), 
                        linestyle=ls_secondary, color=c_secondary, 
                        linewidth=lw_secondary, zorder=zorder_secondary, 
                        alpha=alpha_secondary)
            axs[i,j].axvline(-1, linestyle=ls_secondary, color=c_secondary, 
                        linewidth=lw_secondary, zorder=zorder_secondary,
                        alpha=alpha_secondary)
        if abunds_x[i] == 'AL_FE' and abunds_y[i] == 'MG_MN':
            axs[i,j].axhline(mg_mn_al_fe_halo_line_y[0], linestyle=ls_secondary, 
                        color=c_secondary, linewidth=lw_secondary, 
                        zorder=zorder_secondary, alpha=alpha_secondary)
            axs[i,j].plot([mg_mn_al_fe_halo_line_x[1],mg_mn_al_fe_halo_line_x[2]],
                        [mg_mn_al_fe_halo_line_y[1],mg_mn_al_fe_halo_line_y[2]],
                        linestyle=ls_secondary, color=c_secondary, 
                        linewidth=lw_secondary, zorder=zorder_secondary, 
                        alpha=alpha_secondary)
        if abunds_x[i] == 'FE_H' and abunds_y[i] == 'AL_FE':
            for k in range(len(al_fe_halo_line_x_all)-1):
                axs[i,j].plot([al_fe_halo_line_x_all[k],al_fe_halo_line_x_all[k+1]],
                            [al_fe_halo_line_y_all[k],al_fe_halo_line_y_all[k+1]],
                            linestyle=ls_primary, color='Black', 
                            linewidth=lw_primary, zorder=zorder_primary, 
                            alpha=alpha_primary)
            for k in range(2):
                axs[i,j].plot([al_fe_halo_line_x_acc[k],al_fe_halo_line_x_acc[k+1]],
                            [al_fe_halo_line_y_acc[k],al_fe_halo_line_y_acc[k+1]],
                            linestyle=ls_primary, color='Black', 
                            linewidth=lw_primary, zorder=zorder_primary, 
                            alpha=alpha_primary)
            # Label the spaces
            if j == 0:
                # bbox_props = 
                for k in range(len(halo_type_labels)):     
                    axs[i,j].text(xs_halo_type_label[k], ys_halo_type_label[k],
                                halo_type_labels[k], transform=axs[i,j].transAxes, 
                                fontsize=halo_type_label_fs)
    
        # Remove labels
        if j != 0:
            axs[i,j].set_yticklabels([])
            axs[i,j].set_ylabel('')
        if i == 0:
           axs[i,j].set_xticklabels([])
           axs[i,j].set_xlabel('')
    
        # Add titles for each subplot
        if i == 0:
            # # As a subplot title
            # axs[i,j].set_title(labels[j], fontsize=label_fs)
            
            # As a piece of text within a subplot
            bbox_props = dict(boxstyle='round', facecolor='White', alpha=1.0)
            axs[i,j].text(0.95,0.95, labels[j], fontsize=halo_type_label_fs+2, 
                        transform=axs[i,j].transAxes, va='top', ha='right',
                        bbox=bbox_props)

# Add the margins for each abundances
margin_abunds = ['FE_H', 'AL_FE', 'MG_FE']
margin_kin_space_names = ['eLz','AD','JRLz']
margin_axs = [ax_feh, ax_alfe, ax_mgfe]
margin_lims = [feh_lims, alfe_lims, mgfe_lims]
margin_rot = [False,True,True]
margin_label_top = [False,True,False]
margin_ylabels = ['p([Fe/H])', 'p([Al/Fe])', 'p([Mg/Fe])']
margin_kmask_linewidths = [2.5,2.,1.5]
margin_kmask_zorders = [5,6,7]


for i in range(len(margin_abunds)):

    xa,_ = putil.get_metallicity(allstar_omask[logg_mask],margin_abunds[i])
    kde = scipy.stats.gaussian_kde(xa, bw_method='scott')
    x_plot = np.linspace(margin_lims[i][0], margin_lims[i][1], 100)
    y_plot = kde(x_plot)
    margin_axs[i].tick_params(which='both', direction='in', zorder=9)
    if margin_rot[i]:
        margin_axs[i].plot(y_plot, x_plot, color='Grey', linestyle='-', zorder=3, 
            rasterized=True)
        margin_axs[i].set_xlabel(margin_ylabels[i], fontsize=label_fs-4)
        margin_axs[i].set_ylim(margin_lims[i][0], margin_lims[i][1])
        margin_axs[i].tick_params(labelleft=False, labelbottom=False)
    else:
        margin_axs[i].plot(x_plot, y_plot, color='Grey', linestyle='-', zorder=3, 
            rasterized=True)
        margin_axs[i].set_ylabel(margin_ylabels[i], fontsize=label_fs-4)
        margin_axs[i].set_xlim(margin_lims[i][0], margin_lims[i][1])
        margin_axs[i].tick_params(labelbottom=False, labelleft=False)
    if margin_label_top[i]:
        margin_axs[i].xaxis.set_label_position('top')

    for j in range(n_kin_spaces):
        # Make the kinematic space mask
        ksamp_mask = logg_mask & abund_mask & kmasks[kin_spaces[j]]
        xk,_ = putil.get_metallicity(allstar_omask[ksamp_mask],margin_abunds[i])
        median_xk = np.median(xk)
        print(margin_abunds[i], margin_kin_space_names[j], median_xk)
        kde = scipy.stats.gaussian_kde(xk, bw_method='scott')
        y_plot = kde(x_plot)
        if margin_rot[i]:
            margin_axs[i].plot(y_plot, x_plot, color=colors[j], linestyle='-',
                zorder=margin_kmask_zorders[j], 
                linewidth=margin_kmask_linewidths[j], rasterized=True)
            margin_axs[i].axhline(median_xk, color=colors[j], linestyle='--',
                linewidth=0.5, zorder=4)
        else:
            margin_axs[i].plot(x_plot, y_plot, color=colors[j], linestyle='-',
                zorder=margin_kmask_zorders[j], 
                linewidth=margin_kmask_linewidths[j], rasterized=True)
            margin_axs[i].axvline(median_xk, color=colors[j], linestyle='--',
                linewidth=0.5, zorder=4)

fig.tight_layout()
fig.subplots_adjust(hspace=0.1, wspace=0.1, left=0.09, right=0.9)
fig.savefig(fig_dir+'kinematic_sample_abundances_margins.pdf', dpi=400)
fig.show()

### All three spaces

In [ ]:
# Paramters
n_bins = 50
label_fs = 12
ylabel_pad = -4
cbar_ticklabel_fs = 8
ticklabel_fs = 9
legend_fs = 9

n_abunds = 3
fig = plt.figure(figsize=(textwidth,5))
axs = fig.subplots(nrows=n_abunds, ncols=3)

abunds_x = ['FE_H','FE_H','AL_FE']
abunds_y = ['AL_FE','MG_FE','MG_MN']
abunds_xlabels = ['[Fe/H]', '[Fe/H]', '[Al/Fe]']
abunds_ylabels = ['[Al/Fe]', '[Mg/Fe]', '[Mg/Mn]']
xlims = [ feh_lims, feh_lims, alfe_lims]
ylims = [ alfe_lims, mgfe_lims, mgmn_lims]

# ecc_sort = np.argsort(eELzs_omask[0])
acc_masks = []

for i in range(n_abunds):
    # Abundances
    xa,_ = putil.get_metallicity(allstar_omask[logg_mask],abunds_x[i])
    ya,_ = putil.get_metallicity(allstar_omask[logg_mask],abunds_y[i])

    for j in range(n_kin_spaces):
        # Make the kinematic space mask
        ksamp_mask = logg_mask & abund_mask & kmasks[kin_spaces[j]]
        
        xk,_ = putil.get_metallicity(allstar_omask[ksamp_mask],abunds_x[i])
        yk,_ = putil.get_metallicity(allstar_omask[ksamp_mask],abunds_y[i])

        # Make the accreted mask in the 
        if abunds_x[i]=='FE_H' and abunds_y[i]=='AL_FE':
            acc_mask = putil.lines_boundary_mask(xk, yk,
                al_fe_halo_line_x_acc, al_fe_halo_line_y_acc, where='left')
            acc_masks.append(acc_mask)
            print('Number of stars in kin space, low [Al/Fe]: '+str(kin_spaces[j]), np.sum(acc_mask))
            print('Number of stars in kin space, high [Al/Fe]: '+str(kin_spaces[j]), np.sum(~acc_mask))

        # Plot
        axs[i,j].scatter(xk[acc_masks[j]], yk[acc_masks[j]], c=colors[j],  
            edgecolors='Black', marker='o', s=10, linewidth=0.5, zorder=4, 
            rasterized=True)
        axs[i,j].scatter(xk[~acc_masks[j]], yk[~acc_masks[j]], c='Black', 
            marker='x', s=10, linewidth=1., zorder=5, rasterized=True)
        axs[i,j].scatter(xa, ya, c='Grey', edgecolors='None', marker='o', s=2, 
            zorder=3, rasterized=True)
        
        # Add contours
        xbinsize = (xlims[i][1]-xlims[i][0])/n_bins
        ybinsize = (ylims[i][1]-ylims[i][0])/n_bins
        hist,xedge,yedge = np.histogram2d(xa,ya,
                                          bins=n_bins,
                                          range=[[xlims[i][0]-xbinsize,
                                                  xlims[i][1]+xbinsize],
                                                 [ylims[i][0]-ybinsize,
                                                  ylims[i][1]+ybinsize]])
        xcents = xedge[:-1] + (xlims[i][1]-xlims[i][0])/n_bins
        ycents = yedge[:-1] + (ylims[i][1]-ylims[i][0])/n_bins
        xcnt,ycnt = np.meshgrid(xcents,ycents)
        axs[i,j].contour(xcnt, ycnt, hist.T, colors='Black', 
                         levels=[10**1.75,10**2.25,10**2.75,10**3.25], 
                         zorder=5, alpha=0.5)
        
        axs[i,j].set_xlabel(abunds_xlabels[i], fontsize=label_fs)
        axs[i,j].set_ylabel(abunds_ylabels[i], fontsize=label_fs)
        axs[i,j].set_xlim(xlims[i])
        axs[i,j].set_ylim(ylims[i])
        axs[i,j].tick_params(labelsize=ticklabel_fs)
        
        # Add lines
        if abunds_x[i] == 'FE_H' and abunds_y[i] == 'MG_FE':
            axs[i,j].plot(np.arange(-3,2,1.), 
                        pplot.line_equation(np.arange(-3,2,1.),
                                            alpha_fe_halo_line_m,
                                            alpha_fe_halo_line_b), 
                        linestyle=ls_secondary, color=c_secondary, 
                        linewidth=lw_secondary, zorder=zorder_secondary, 
                        alpha=alpha_secondary)
            axs[i,j].axvline(-1, linestyle=ls_secondary, color=c_secondary, 
                        linewidth=lw_secondary, zorder=zorder_secondary,
                        alpha=alpha_secondary)
        if abunds_x[i] == 'AL_FE' and abunds_y[i] == 'MG_MN':
            axs[i,j].axhline(mg_mn_al_fe_halo_line_y[0], linestyle=ls_secondary, 
                        color=c_secondary, linewidth=lw_secondary, 
                        zorder=zorder_secondary, alpha=alpha_secondary)
            axs[i,j].plot([mg_mn_al_fe_halo_line_x[1],mg_mn_al_fe_halo_line_x[2]],
                        [mg_mn_al_fe_halo_line_y[1],mg_mn_al_fe_halo_line_y[2]],
                        linestyle=ls_secondary, color=c_secondary, 
                        linewidth=lw_secondary, zorder=zorder_secondary, 
                        alpha=alpha_secondary)
        if abunds_x[i] == 'FE_H' and abunds_y[i] == 'AL_FE':
            for k in range(len(al_fe_halo_line_x_all)-1):
                axs[i,j].plot([al_fe_halo_line_x_all[k],al_fe_halo_line_x_all[k+1]],
                            [al_fe_halo_line_y_all[k],al_fe_halo_line_y_all[k+1]],
                            linestyle=ls_primary, color='Black', 
                            linewidth=lw_primary, zorder=zorder_primary, 
                            alpha=alpha_primary)
            for k in range(2):
                axs[i,j].plot([al_fe_halo_line_x_acc[k],al_fe_halo_line_x_acc[k+1]],
                            [al_fe_halo_line_y_acc[k],al_fe_halo_line_y_acc[k+1]],
                            linestyle=ls_primary, color='Black', 
                            linewidth=lw_primary, zorder=zorder_primary, 
                            alpha=alpha_primary)
            # Label the spaces
            if j == 0:
                for k in range(len(halo_type_labels)):     
                    axs[i,j].text(xs_halo_type_label[k], ys_halo_type_label[k],
                                halo_type_labels[k], transform=axs[i,j].transAxes, 
                                fontsize=halo_type_label_fs)
    
        # Remove labels
        if j != 0:
            axs[i,j].set_yticklabels([])
            axs[i,j].set_ylabel('')
        #if i == 0:
        #    axs[i,j].set_xticklabels([])
        #    axs[i,j].set_xlabel('')
    
        # Add titles for each subplot
        if i == 0:
            axs[i,j].set_title(labels[j], fontsize=label_fs)
            # axs[i,j].text(0.7,0.95, labels[j], fontsize=20, 
            #             transform=axs[i,j].transAxes, va='top', ha='left')

fig.tight_layout()
fig.subplots_adjust(hspace=0.4, wspace=0.2, left=0.09, right=0.9)
#fig.savefig(fig_dir+'kinematic_sample_abundances_3panel.pdf', dpi=400)
fig.show()

### Plot chemistry of the kinematic selections but only in the [Al/Fe]-[Fe/H] space

In [ ]:
# # Paramters
# n_bins = 50

# fig = plt.figure(figsize=(12,4))
# axs = fig.subplots(nrows=1, ncols=3).flatten()

# feh,_ = putil.get_metallicity(allstar_omask,'FE_H')
# alfe,_ = putil.get_metallicity(allstar_omask,'AL_FE')
# xlabel = '[Fe/H]'
# ylabel = '[Al/Fe]'
# xlims = [-2.5,0.75]
# ylims = [-0.6,0.6]

# colors = ['Red','DarkOrange','DodgerBlue']
# labels = ['$e-L_\mathrm{z}$','AD','$\sqrt{J_\mathrm{R}}-L_\mathrm{z}$']
# for i in range(n_kin_spaces):
#     # Make the kinematic space mask
#     ksamp_mask = logg_mask & abund_mask & kmasks[kin_spaces[i]]

#     # Make the accreted mask
#     acc_mask = putil.lines_boundary_mask(feh[ksamp_mask], alfe[ksamp_mask],
#         al_fe_halo_line_x_acc, al_fe_halo_line_y_acc, where='left')
    
#     # Plot
#     axs[i].scatter(feh[ksamp_mask][acc_mask], alfe[ksamp_mask][acc_mask],
#                    c=colors[i],  edgecolors='Black', marker='o', s=10, 
#                    zorder=4, rasterized=True)
#     axs[i].scatter(feh[ksamp_mask][~acc_mask], alfe[ksamp_mask][~acc_mask],
#                    c='Black', marker='x', s=10, 
#                    zorder=4, rasterized=True)
#     axs[i].scatter(feh[~ksamp_mask], alfe[~ksamp_mask], 
#                    c='Grey', 
#                    edgecolors='None', marker='o', s=2, 
#                    zorder=3, rasterized=True)
    
#     # Add contours
#     xbinsize = (xlims[1]-xlims[0])/n_bins
#     ybinsize = (ylims[1]-ylims[0])/n_bins
#     hist,xedge,yedge = np.histogram2d(feh[~ksamp_mask],alfe[~ksamp_mask],
#                                       bins=n_bins,
#                                       range=[[xlims[0]-xbinsize,
#                                               xlims[1]+xbinsize],
#                                              [ylims[0]-ybinsize,
#                                               ylims[1]+ybinsize]])
#     xcents = xedge[:-1] + (xlims[1]-xlims[0])/n_bins
#     ycents = yedge[:-1] + (ylims[1]-ylims[0])/n_bins
#     xcnt,ycnt = np.meshgrid(xcents,ycents)
#     axs[i].contour(xcnt, ycnt, hist.T, colors='Black', levels=[50,100,500], 
#                    zorder=5, alpha=0.5)
    
#     axs[i].set_xlabel(xlabel, fontsize=20)
#     axs[i].set_ylabel(ylabel, fontsize=20)
#     axs[i].set_xlim(xlims)
#     axs[i].set_ylim(ylims)
    
#     # Add lines
#     for j in range(len(al_fe_halo_line_x_all)-1):
#         axs[i].plot([al_fe_halo_line_x_all[j],al_fe_halo_line_x_all[j+1]],
#                     [al_fe_halo_line_y_all[j],al_fe_halo_line_y_all[j+1]],
#                     linestyle='dashed', color='Black', linewidth=2., 
#                     zorder=6)
#     for j in range(2):
#         axs[i].plot([al_fe_halo_line_x_acc[j],al_fe_halo_line_x_acc[j+1]],
#                     [al_fe_halo_line_y_acc[j],al_fe_halo_line_y_acc[j+1]],
#                     linestyle='dashed', color='Black', linewidth=2., 
#                     zorder=6)

#     for k in range(len(al_fe_halo_line_x_all)-1):
#         axs[i].plot([al_fe_halo_line_x_all[k],al_fe_halo_line_x_all[k+1]],
#                     [al_fe_halo_line_y_all[k],al_fe_halo_line_y_all[k+1]],
#                     linestyle=ls_primary, color='Black', 
#                     linewidth=lw_primary, zorder=zorder_primary, 
#                     alpha=alpha_primary)
#     for k in range(2):
#         axs[i].plot([al_fe_halo_line_x_acc[k],al_fe_halo_line_x_acc[k+1]],
#                     [al_fe_halo_line_y_acc[k],al_fe_halo_line_y_acc[k+1]],
#                     linestyle=ls_primary, color='Black', 
#                     linewidth=lw_primary, zorder=zorder_primary, 
#                     alpha=alpha_primary)
#     # Label the spaces
#     for k in range(len(halo_type_labels)):     
#         axs[i].text(xs_halo_type_label[k], ys_halo_type_label[k],
#                     halo_type_labels[k], transform=axs[i].transAxes, 
#                     fontsize=halo_type_label_fs)
    
#     # Remove labels
#     if i != 0:
#         axs[i].set_yticklabels([])
#         axs[i].set_ylabel('')
    
#     # Add titles for each subplot
#     axs[i].text(0.05,0.95, labels[i], fontsize=20, transform=axs[i].transAxes,
#                 va='top', ha='left')
    
# fig.tight_layout()
# # fig.subplots_adjust(hspace=0.35, wspace=0.35, left=0.09, right=0.9)
# fig.savefig(fig_dir+'kinematic_sample_al_fe.pdf')
# fig.show()

### Find the overlap between the various kinematic samples

In [ ]:
# Get the accreted mask
hmask = logg_mask & abund_mask
xk,_ = putil.get_metallicity(allstar_omask,'FE_H')
yk,_ = putil.get_metallicity(allstar_omask,'AL_FE')
accmask = putil.lines_boundary_mask(xk, yk,
    al_fe_halo_line_x_acc, al_fe_halo_line_y_acc, where='left')

In [ ]:
# Count the number of stars in the overlapping regions
for i in range(3):
    ksamp_mask = hmask & accmask & kmasks[kin_spaces[i]]
    print(kin_spaces[i], ' number of stars: ', np.sum(ksamp_mask))
print('')

kin_spaces_combinations = [['eLz','AD'],
                           ['eLz','JRLz'],
                           ['AD','JRLz']]
kin_spaces_combinations_labels = ['eLz + AD', 'eLz + JRLz', 'AD + JRLz']
for i in range(3):
    ksamp_mask_combo = hmask & accmask & kmasks[kin_spaces_combinations[i][0]] & kmasks[kin_spaces_combinations[i][1]]
    print(kin_spaces_combinations_labels[i], ' number of stars: ', np.sum(ksamp_mask_combo))
print('')

kin_spaces_exc_combinations = [['AD','JRLz'],
                               ['eLz','JRLz'],
                               ['eLz','AD']]
for i in range(3):
    ksamp_mask_exc = hmask & accmask & kmasks[kin_spaces[i]] & \
        ~kmasks[kin_spaces_exc_combinations[i][0]] & \
        ~kmasks[kin_spaces_exc_combinations[i][1]]
    print(kin_spaces[i], ' exclusive number of stars: ', np.sum(ksamp_mask_exc))

In [ ]:
# Paramters
n_bins = 50
label_fs = 12
ylabel_pad = -2
cbar_ticklabel_fs = 8
ticklabel_fs = 9
legend_fs = 9

n_abunds = 2
fig = plt.figure(figsize=(textwidth,6))
gs = fig.add_gridspec(nrows=n_abunds+1, ncols=2, width_ratios=[1,0.33],
    height_ratios=[0.33,1,1])
axs = np.empty((n_abunds), dtype=object)
axs[0] = fig.add_subplot(gs[1,0])
axs[1] = fig.add_subplot(gs[2,0])
ax_feh = fig.add_subplot(gs[0,0])
ax_alfe = fig.add_subplot(gs[1,-1])
ax_mgfe = fig.add_subplot(gs[2,-1])  

abunds_x = ['FE_H','FE_H']
abunds_y = ['AL_FE','MG_FE']
abunds_xlabels = ['[Fe/H]', '[Fe/H]']
abunds_ylabels = ['[Al/Fe]', '[Mg/Fe]']
xlims = [ feh_lims, feh_lims]
ylims = [ alfe_lims, mgfe_lims]

# ecc_sort = np.argsort(eELzs_omask[0])
_kin_space = 'eLz'
_kin_space_combinations = ['eLz', 'AD']
_kin_space_exc_combinations = ['AD', 'JRLz']
_kin_space_indx = 0

# Make the kinematic space mask
ksamp_mask = hmask & accmask & kmasks[_kin_space]
ksamp_mask_combo = hmask & accmask & \
    kmasks[_kin_space_combinations[0]] & \
    kmasks[_kin_space_combinations[1]]
ksamp_mask_exc = hmask & accmask & kmasks[_kin_space] & \
    ~kmasks[_kin_space_exc_combinations[0]] & \
    ~kmasks[_kin_space_exc_combinations[1]]

for i in range(n_abunds):
    # Abundances
    xa,_ = putil.get_metallicity(allstar_omask[logg_mask],abunds_x[i])
    ya,_ = putil.get_metallicity(allstar_omask[logg_mask],abunds_y[i])
    
    xk,_ = putil.get_metallicity(allstar_omask[ksamp_mask],abunds_x[i])
    yk,_ = putil.get_metallicity(allstar_omask[ksamp_mask],abunds_y[i])

    xexc,_ = putil.get_metallicity(allstar_omask[ksamp_mask_exc],abunds_x[i])
    yexc,_ = putil.get_metallicity(allstar_omask[ksamp_mask_exc],abunds_y[i])

    # Plot
    axs[i].scatter(xk, yk, c='Black',  
        edgecolors='Black', marker='x', s=20, linewidth=0.5, zorder=4, 
        rasterized=True)
    axs[i].scatter(xexc, yexc, c=colors[_kin_space_indx],  
        edgecolors='Black', marker='o', s=20, linewidth=0.5, zorder=5, 
        rasterized=True)
    axs[i].scatter(xa, ya, c='Grey', edgecolors='None', marker='o', s=2, 
        zorder=3, rasterized=True)
        
    # Add contours
    xbinsize = (xlims[i][1]-xlims[i][0])/n_bins
    ybinsize = (ylims[i][1]-ylims[i][0])/n_bins
    hist,xedge,yedge = np.histogram2d(xa,ya,
                                        bins=n_bins,
                                        range=[[xlims[i][0]-xbinsize,
                                                xlims[i][1]+xbinsize],
                                                [ylims[i][0]-ybinsize,
                                                ylims[i][1]+ybinsize]])
    xcents = xedge[:-1] + (xlims[i][1]-xlims[i][0])/n_bins
    ycents = yedge[:-1] + (ylims[i][1]-ylims[i][0])/n_bins
    xcnt,ycnt = np.meshgrid(xcents,ycents)
    axs[i].contour(xcnt, ycnt, hist.T, colors='Black', 
                        levels=[10**1.75,10**2.25,10**2.75,10**3.25], 
                        zorder=5, alpha=0.5)
        
    axs[i].set_xlabel(abunds_xlabels[i], fontsize=label_fs)
    axs[i].set_ylabel(abunds_ylabels[i], fontsize=label_fs)
    axs[i].set_xlim(xlims[i])
    axs[i].set_ylim(ylims[i])
    axs[i].tick_params(labelsize=ticklabel_fs)
        
    # Add lines
    if abunds_x[i] == 'FE_H' and abunds_y[i] == 'MG_FE':
        axs[i].plot(np.arange(-3,2,1.), 
                    pplot.line_equation(np.arange(-3,2,1.),
                                        alpha_fe_halo_line_m,
                                        alpha_fe_halo_line_b), 
                    linestyle=ls_secondary, color=c_secondary, 
                    linewidth=lw_secondary, zorder=zorder_secondary, 
                    alpha=alpha_secondary)
        axs[i].axvline(-1, linestyle=ls_secondary, color=c_secondary, 
                    linewidth=lw_secondary, zorder=zorder_secondary,
                    alpha=alpha_secondary)
    if abunds_x[i] == 'AL_FE' and abunds_y[i] == 'MG_MN':
        axs[i].axhline(mg_mn_al_fe_halo_line_y[0], linestyle=ls_secondary, 
                    color=c_secondary, linewidth=lw_secondary, 
                    zorder=zorder_secondary, alpha=alpha_secondary)
        axs[i].plot([mg_mn_al_fe_halo_line_x[1],mg_mn_al_fe_halo_line_x[2]],
                    [mg_mn_al_fe_halo_line_y[1],mg_mn_al_fe_halo_line_y[2]],
                    linestyle=ls_secondary, color=c_secondary, 
                    linewidth=lw_secondary, zorder=zorder_secondary, 
                    alpha=alpha_secondary)
    if abunds_x[i] == 'FE_H' and abunds_y[i] == 'AL_FE':
        for k in range(len(al_fe_halo_line_x_all)-1):
            axs[i].plot([al_fe_halo_line_x_all[k],al_fe_halo_line_x_all[k+1]],
                        [al_fe_halo_line_y_all[k],al_fe_halo_line_y_all[k+1]],
                        linestyle=ls_primary, color='Black', 
                        linewidth=lw_primary, zorder=zorder_primary, 
                        alpha=alpha_primary)
        for k in range(2):
            axs[i].plot([al_fe_halo_line_x_acc[k],al_fe_halo_line_x_acc[k+1]],
                        [al_fe_halo_line_y_acc[k],al_fe_halo_line_y_acc[k+1]],
                        linestyle=ls_primary, color='Black', 
                        linewidth=lw_primary, zorder=zorder_primary, 
                        alpha=alpha_primary)
        # # Label the spaces
        # if j == 0:
        #     for k in range(len(halo_type_labels)):     
        #         axs[i,j].text(xs_halo_type_label[k], ys_halo_type_label[k],
        #                     halo_type_labels[k], transform=axs[i,j].transAxes, 
        #                     fontsize=halo_type_label_fs)
    
        # Remove labels
        # if j != 0:
        #     axs[i].set_yticklabels([])
        #     axs[i].set_ylabel('')
        if i == 0:
           axs[i].set_xticklabels([])
           axs[i].set_xlabel('')
    
        # Add titles for each subplot
        if i == 0:
            # # As a subplot title
            # axs[i,j].set_title(labels[j], fontsize=label_fs)
            
            # As a piece of text within a subplot
            bbox_props = dict(boxstyle='round', facecolor='White', alpha=1.0)
            axs[i].text(0.95,0.95, labels[_kin_space_indx], fontsize=halo_type_label_fs+2, 
                        transform=axs[i].transAxes, va='top', ha='right',
                        bbox=bbox_props)

# Add the margins for each abundances
margin_abunds = ['FE_H', 'AL_FE', 'MG_FE']
margin_axs = [ax_feh, ax_alfe, ax_mgfe]
margin_lims = [feh_lims, alfe_lims, mgfe_lims]
margin_rot = [False,True,True]
margin_label_top = [False,True,False]
margin_ylabels = ['p([Fe/H])', 'p([Al/Fe])', 'p([Mg/Fe])']
margin_kmask_linewidths = [2.5,2.,1.5]
margin_kmask_zorders = [5,6,7]


for i in range(len(margin_abunds)):

    xa,_ = putil.get_metallicity(allstar_omask[logg_mask],margin_abunds[i])
    kde = scipy.stats.gaussian_kde(xa, bw_method='scott')
    x_plot = np.linspace(margin_lims[i][0], margin_lims[i][1], 100)
    y_plot = kde(x_plot)
    margin_axs[i].tick_params(which='both', direction='in', zorder=9)
    if margin_rot[i]:
        margin_axs[i].plot(y_plot, x_plot, color='Grey', linestyle='-', zorder=3, 
            rasterized=True)
        margin_axs[i].set_xlabel(margin_ylabels[i], fontsize=label_fs-4)
        margin_axs[i].set_ylim(margin_lims[i][0], margin_lims[i][1])
        margin_axs[i].tick_params(labelleft=False, labelbottom=False)
    else:
        margin_axs[i].plot(x_plot, y_plot, color='Grey', linestyle='-', zorder=3, 
            rasterized=True)
        margin_axs[i].set_ylabel(margin_ylabels[i], fontsize=label_fs-4)
        margin_axs[i].set_xlim(margin_lims[i][0], margin_lims[i][1])
        margin_axs[i].tick_params(labelbottom=False, labelleft=False)
    if margin_label_top[i]:
        margin_axs[i].xaxis.set_label_position('top')

    # Make the kinematic space mask for the whole kinematic sample
    # ksamp_mask = logg_mask & abund_mask & kmasks[_kin_space]
    xk,_ = putil.get_metallicity(allstar_omask[ksamp_mask],margin_abunds[i])
    median_xk = np.median(xk)
    print('common', margin_abunds[i], margin_kin_space_names[_kin_space_indx], median_xk)
    kde = scipy.stats.gaussian_kde(xk, bw_method='scott')
    y_plot = kde(x_plot)
    if margin_rot[i]:
        margin_axs[i].plot(y_plot, x_plot, color='Black', linestyle='-',
            zorder=margin_kmask_zorders[j], 
            linewidth=margin_kmask_linewidths[j], rasterized=True)
        margin_axs[i].axhline(median_xk, color='Black', linestyle='-',
            linewidth=1, zorder=4)
    else:
        margin_axs[i].plot(x_plot, y_plot, color='Black', linestyle='-',
            zorder=margin_kmask_zorders[j], 
            linewidth=margin_kmask_linewidths[j], rasterized=True)
        margin_axs[i].axvline(median_xk, color='Black', linestyle='-',
            linewidth=1, zorder=4)
    
    # Make the kinematic space mask for the exclusive kinematic
    # ksamp_mask = logg_mask & abund_mask & kmasks[_kin_space]
    xexc,_ = putil.get_metallicity(allstar_omask[ksamp_mask_exc],margin_abunds[i])
    median_xk = np.median(xexc)
    print('exclusive', margin_abunds[i], margin_kin_space_names[_kin_space_indx], median_xk)
    kde = scipy.stats.gaussian_kde(xexc, bw_method='scott')
    y_plot = kde(x_plot)
    if margin_rot[i]:
        margin_axs[i].plot(y_plot, x_plot, color=colors[_kin_space_indx], linestyle='-',
            zorder=margin_kmask_zorders[j], 
            linewidth=margin_kmask_linewidths[j], rasterized=True)
        margin_axs[i].axhline(median_xk, color=colors[_kin_space_indx], linestyle='-',
            linewidth=1, zorder=4)
    else:
        margin_axs[i].plot(x_plot, y_plot, color=colors[_kin_space_indx], linestyle='-',
            zorder=margin_kmask_zorders[j], 
            linewidth=margin_kmask_linewidths[j], rasterized=True)
        margin_axs[i].axvline(median_xk, color=colors[_kin_space_indx], linestyle='-',
            linewidth=1, zorder=4)

fig.tight_layout()
fig.subplots_adjust(hspace=0.1, wspace=0.1, left=0.09, right=0.9)
# fig.savefig(fig_dir+'kinematic_sample_abundances.pdf', dpi=400)
fig.show()

In [ ]:
# Paramters
n_bins = 50
label_fs = 12
ylabel_pad = -2
cbar_ticklabel_fs = 8
ticklabel_fs = 9
legend_fs = 9

n_abunds = 2
fig = plt.figure(figsize=(textwidth,6))
gs = fig.add_gridspec(nrows=n_abunds+1, ncols=2, width_ratios=[1,0.33],
    height_ratios=[0.33,1,1])
axs = np.empty((n_abunds), dtype=object)
axs[0] = fig.add_subplot(gs[1,0])
axs[1] = fig.add_subplot(gs[2,0])
ax_feh = fig.add_subplot(gs[0,0])
ax_alfe = fig.add_subplot(gs[1,-1])
ax_mgfe = fig.add_subplot(gs[2,-1])  

abunds_x = ['FE_H','FE_H']
abunds_y = ['AL_FE','MG_FE']
abunds_xlabels = ['[Fe/H]', '[Fe/H]']
abunds_ylabels = ['[Al/Fe]', '[Mg/Fe]']
xlims = [ feh_lims, feh_lims]
ylims = [ alfe_lims, mgfe_lims]

# ecc_sort = np.argsort(eELzs_omask[0])
_kin_space = 'JRLz'
# _kin_space_combinations = ['eLz', 'AD']
_kin_space_exc_combinations = ['eLz', 'AD']
_kin_space_indx = 2

# Make the kinematic space mask
ksamp_mask = hmask & accmask & kmasks[_kin_space]
ksamp_mask_combo = hmask & accmask & \
    kmasks[_kin_space_combinations[0]] & \
    kmasks[_kin_space_combinations[1]]
ksamp_mask_exc = hmask & accmask & kmasks[_kin_space] & \
    ~kmasks[_kin_space_exc_combinations[0]] & \
    ~kmasks[_kin_space_exc_combinations[1]]

for i in range(n_abunds):
    # Abundances
    xa,_ = putil.get_metallicity(allstar_omask[logg_mask],abunds_x[i])
    ya,_ = putil.get_metallicity(allstar_omask[logg_mask],abunds_y[i])
    
    xk,_ = putil.get_metallicity(allstar_omask[ksamp_mask],abunds_x[i])
    yk,_ = putil.get_metallicity(allstar_omask[ksamp_mask],abunds_y[i])

    xexc,_ = putil.get_metallicity(allstar_omask[ksamp_mask_exc],abunds_x[i])
    yexc,_ = putil.get_metallicity(allstar_omask[ksamp_mask_exc],abunds_y[i])

    # Plot
    axs[i].scatter(xk, yk, c='Black',  
        edgecolors='Black', marker='x', s=20, linewidth=0.5, zorder=4, 
        rasterized=True)
    axs[i].scatter(xexc, yexc, c=colors[_kin_space_indx],  
        edgecolors='Black', marker='o', s=20, linewidth=0.5, zorder=5, 
        rasterized=True)
    axs[i].scatter(xa, ya, c='Grey', edgecolors='None', marker='o', s=2, 
        zorder=3, rasterized=True)
        
    # Add contours
    xbinsize = (xlims[i][1]-xlims[i][0])/n_bins
    ybinsize = (ylims[i][1]-ylims[i][0])/n_bins
    hist,xedge,yedge = np.histogram2d(xa,ya,
                                        bins=n_bins,
                                        range=[[xlims[i][0]-xbinsize,
                                                xlims[i][1]+xbinsize],
                                                [ylims[i][0]-ybinsize,
                                                ylims[i][1]+ybinsize]])
    xcents = xedge[:-1] + (xlims[i][1]-xlims[i][0])/n_bins
    ycents = yedge[:-1] + (ylims[i][1]-ylims[i][0])/n_bins
    xcnt,ycnt = np.meshgrid(xcents,ycents)
    axs[i].contour(xcnt, ycnt, hist.T, colors='Black', 
                        levels=[10**1.75,10**2.25,10**2.75,10**3.25], 
                        zorder=5, alpha=0.5)
        
    axs[i].set_xlabel(abunds_xlabels[i], fontsize=label_fs)
    axs[i].set_ylabel(abunds_ylabels[i], fontsize=label_fs)
    axs[i].set_xlim(xlims[i])
    axs[i].set_ylim(ylims[i])
    axs[i].tick_params(labelsize=ticklabel_fs)
        
    # Add lines
    if abunds_x[i] == 'FE_H' and abunds_y[i] == 'MG_FE':
        axs[i].plot(np.arange(-3,2,1.), 
                    pplot.line_equation(np.arange(-3,2,1.),
                                        alpha_fe_halo_line_m,
                                        alpha_fe_halo_line_b), 
                    linestyle=ls_secondary, color=c_secondary, 
                    linewidth=lw_secondary, zorder=zorder_secondary, 
                    alpha=alpha_secondary)
        axs[i].axvline(-1, linestyle=ls_secondary, color=c_secondary, 
                    linewidth=lw_secondary, zorder=zorder_secondary,
                    alpha=alpha_secondary)
    if abunds_x[i] == 'AL_FE' and abunds_y[i] == 'MG_MN':
        axs[i].axhline(mg_mn_al_fe_halo_line_y[0], linestyle=ls_secondary, 
                    color=c_secondary, linewidth=lw_secondary, 
                    zorder=zorder_secondary, alpha=alpha_secondary)
        axs[i].plot([mg_mn_al_fe_halo_line_x[1],mg_mn_al_fe_halo_line_x[2]],
                    [mg_mn_al_fe_halo_line_y[1],mg_mn_al_fe_halo_line_y[2]],
                    linestyle=ls_secondary, color=c_secondary, 
                    linewidth=lw_secondary, zorder=zorder_secondary, 
                    alpha=alpha_secondary)
    if abunds_x[i] == 'FE_H' and abunds_y[i] == 'AL_FE':
        for k in range(len(al_fe_halo_line_x_all)-1):
            axs[i].plot([al_fe_halo_line_x_all[k],al_fe_halo_line_x_all[k+1]],
                        [al_fe_halo_line_y_all[k],al_fe_halo_line_y_all[k+1]],
                        linestyle=ls_primary, color='Black', 
                        linewidth=lw_primary, zorder=zorder_primary, 
                        alpha=alpha_primary)
        for k in range(2):
            axs[i].plot([al_fe_halo_line_x_acc[k],al_fe_halo_line_x_acc[k+1]],
                        [al_fe_halo_line_y_acc[k],al_fe_halo_line_y_acc[k+1]],
                        linestyle=ls_primary, color='Black', 
                        linewidth=lw_primary, zorder=zorder_primary, 
                        alpha=alpha_primary)
        # # Label the spaces
        # if j == 0:
        #     for k in range(len(halo_type_labels)):     
        #         axs[i,j].text(xs_halo_type_label[k], ys_halo_type_label[k],
        #                     halo_type_labels[k], transform=axs[i,j].transAxes, 
        #                     fontsize=halo_type_label_fs)
    
        # Remove labels
        # if j != 0:
        #     axs[i].set_yticklabels([])
        #     axs[i].set_ylabel('')
        if i == 0:
           axs[i].set_xticklabels([])
           axs[i].set_xlabel('')
    
        # Add titles for each subplot
        if i == 0:
            # # As a subplot title
            # axs[i,j].set_title(labels[j], fontsize=label_fs)
            
            # As a piece of text within a subplot
            bbox_props = dict(boxstyle='round', facecolor='White', alpha=1.0)
            axs[i].text(0.95,0.95, labels[_kin_space_indx], fontsize=halo_type_label_fs+2, 
                        transform=axs[i].transAxes, va='top', ha='right',
                        bbox=bbox_props)

# Add the margins for each abundances
margin_abunds = ['FE_H', 'AL_FE', 'MG_FE']
margin_axs = [ax_feh, ax_alfe, ax_mgfe]
margin_lims = [feh_lims, alfe_lims, mgfe_lims]
margin_rot = [False,True,True]
margin_label_top = [False,True,False]
margin_ylabels = ['p([Fe/H])', 'p([Al/Fe])', 'p([Mg/Fe])']
margin_kmask_linewidths = [2.5,2.,1.5]
margin_kmask_zorders = [5,6,7]


for i in range(len(margin_abunds)):

    xa,_ = putil.get_metallicity(allstar_omask[logg_mask],margin_abunds[i])
    kde = scipy.stats.gaussian_kde(xa, bw_method='scott')
    x_plot = np.linspace(margin_lims[i][0], margin_lims[i][1], 100)
    y_plot = kde(x_plot)
    margin_axs[i].tick_params(which='both', direction='in', zorder=9)
    if margin_rot[i]:
        margin_axs[i].plot(y_plot, x_plot, color='Grey', linestyle='-', zorder=3, 
            rasterized=True)
        margin_axs[i].set_xlabel(margin_ylabels[i], fontsize=label_fs-4)
        margin_axs[i].set_ylim(margin_lims[i][0], margin_lims[i][1])
        margin_axs[i].tick_params(labelleft=False, labelbottom=False)
    else:
        margin_axs[i].plot(x_plot, y_plot, color='Grey', linestyle='-', zorder=3, 
            rasterized=True)
        margin_axs[i].set_ylabel(margin_ylabels[i], fontsize=label_fs-4)
        margin_axs[i].set_xlim(margin_lims[i][0], margin_lims[i][1])
        margin_axs[i].tick_params(labelbottom=False, labelleft=False)
    if margin_label_top[i]:
        margin_axs[i].xaxis.set_label_position('top')

    # Make the kinematic space mask for the whole kinematic sample
    # ksamp_mask = logg_mask & abund_mask & kmasks[_kin_space]
    xk,_ = putil.get_metallicity(allstar_omask[ksamp_mask],margin_abunds[i])
    median_xk = np.median(xk)
    print('common', margin_abunds[i], margin_kin_space_names[_kin_space_indx], median_xk)
    kde = scipy.stats.gaussian_kde(xk, bw_method='scott')
    y_plot = kde(x_plot)
    if margin_rot[i]:
        margin_axs[i].plot(y_plot, x_plot, color='Black', linestyle='-',
            zorder=margin_kmask_zorders[j], 
            linewidth=margin_kmask_linewidths[j], rasterized=True)
        margin_axs[i].axhline(median_xk, color='Black', linestyle='-',
            linewidth=1, zorder=4)
    else:
        margin_axs[i].plot(x_plot, y_plot, color='Black', linestyle='-',
            zorder=margin_kmask_zorders[j], 
            linewidth=margin_kmask_linewidths[j], rasterized=True)
        margin_axs[i].axvline(median_xk, color='Black', linestyle='-',
            linewidth=1, zorder=4)
    
    # Make the kinematic space mask for the exclusive kinematic
    # ksamp_mask = logg_mask & abund_mask & kmasks[_kin_space]
    xexc,_ = putil.get_metallicity(allstar_omask[ksamp_mask_exc],margin_abunds[i])
    median_xk = np.median(xexc)
    print('exclusive', margin_abunds[i], margin_kin_space_names[_kin_space_indx], median_xk)
    kde = scipy.stats.gaussian_kde(xexc, bw_method='scott')
    y_plot = kde(x_plot)
    if margin_rot[i]:
        margin_axs[i].plot(y_plot, x_plot, color=colors[_kin_space_indx], linestyle='-',
            zorder=margin_kmask_zorders[j], 
            linewidth=margin_kmask_linewidths[j], rasterized=True)
        margin_axs[i].axhline(median_xk, color=colors[_kin_space_indx], linestyle='-',
            linewidth=1, zorder=4)
    else:
        margin_axs[i].plot(x_plot, y_plot, color=colors[_kin_space_indx], linestyle='-',
            zorder=margin_kmask_zorders[j], 
            linewidth=margin_kmask_linewidths[j], rasterized=True)
        margin_axs[i].axvline(median_xk, color=colors[_kin_space_indx], linestyle='-',
            linewidth=1, zorder=4)

fig.tight_layout()
fig.subplots_adjust(hspace=0.1, wspace=0.1, left=0.09, right=0.9)
# fig.savefig(fig_dir+'kinematic_sample_abundances.pdf', dpi=400)
fig.show()